In [ ]:
pip install mapclassify

In [ ]:
# Importing required modules for mapping, creating path variable
import mapclassify
import shapely
import pandas as pd
import geopandas as gpd
import folium
import os

path = os.getcwd()

In [ ]:
# Dictionary mapping associations to boundary descriptions
desc_dict = {
    'FWSA': 'Federal Way School District 210',
    'SCSA': 'White River, Tahoma, Auburn, Kent, and Enumclaw School Districts',
    'EYSA': '''The Eastside Youth Soccer Association ("EYSA")  has jurisdiction over youth soccer within the territory
     consisting of Mercer Island and a portion of King County lying east of Lake Washington as granted by the Washington
     State Youth Soccer Association ("WSYSA").
     Schools not included: Apollo, Briarwood, Maple Hills, Newcastle, Maywood, Liberty.''',
    'SNVYSA': 'Snoqualmie Valley & Riverview School Districts',
    'LWYSA': "LWYSA's boundaries cover the Lake Washington School District Area, including Kirkland, Juanita, Redmond, Union Hill and Sammamish.",
    'PCSA': 'Pierce County, without White River School District',
    'NCYSA': 'NCYSA shall operate in Snohomish County and with Skagit and Island Counties through intertie agreements.',
    'LCYSA': 'Lewis County',
    'CYSA': 'Cowlitz County',
    'GHFYSA': '''GHFYSA shall have as its geography of operations all of Grays Harbor County WA, the northern portion
    of Pacific County WA, and southern Mason County WA.''',
    'TCYSA': 'Thurston County',
    'WCYSA': 'Whatcom County',
    'SKVYSA': 'SKVYSA is the parent/sanctioning organization for most USSF youth soccer within the Skagit Valley and San Juan Islands.',
    'NSYSA': 'Kitsap County, North Mason County, and Bainbridge Island',
    'SWYSA': 'Clark County',
    'KVJSA': 'Kittitas County',
    'SYSA': 'Seattle and Shoreline School Districts',
    'NYSA': 'Northshore School District + Edmonds School District + Maltby Elementary + 60 Acres Agreement',
    'GRTYSA': '''All of the Renton School District except that the northern boundaries of the Renton area of control shall be the
Renton City Limits along SE 68th St. to 114th SE, then southeast to 121st St. SE, then east along SE 78th St., to a
point of Squak Mt., then follow to the eastern boundary of the Issaquah School District. The southern half of the
Issaquah School District is included in the GRTYSA area of control, plus the area within the Kent School District
that lies east of 140th SE and north of Petrovitsky road, otherwise known as the Fairwood area. The western
boundaries shall be the Tukwila School District''',
    'HSA': '''The membership of the HSA shall consist of incorporated and un-incorporated organizations engaged in
youth soccer within the Highline School District and the West Seattle area, west of the Duwamish
River''',
    'WIYSA': 'Whidbey Island',
    'SnoYSA': 'Snohomish School District 201',
    'SSCYSA': 'Edmonds School District 15',
    'OYSA': '''(OYSA) provides organization to the Youth Soccer program in Washington State in and around the Sequim, Port Angeles and unincorporated areas
       on the Olympic Peninsula.'''
}

In [ ]:
# Due to GeoPandas lacking a .map() function, manually creating a column
# here for the boundary descriptions for each association
clean_geo = gpd.read_file(path + '/Ver 2.2.json')

desc = []
for item in clean_geo['NAME']:
  desc.append(desc_dict[item])

clean_geo['Boundary Description'] = desc

In [ ]:
def make_point(info):
  lon = info['Longitude']
  lat = info['Latitude']
  return shapely.Point(lon, lat)

In [ ]:
def sound_FC_using(name):
  fields=['Meadowdale Playfields',
          'Meadowdale High School',
          'Edmonds Community College',
          'Edmonds Stadium',
          'Evergreen Playfield',
          'Ballinger Park',
          'Terrace Park Elementary',
          'Mountlake Terrace High School',
          'Lynnwood High School'
          ]

  return 'red' if name in fields else 'blue'

In [42]:
# Creating a map centered on desired region (WA State)
assoc_map = folium.Map(location=[47.6061, -121.7401], zoom_start=7, tiles='CartoDB positron')

# Adding association zoning layer
clean_geo.explore(
    m=assoc_map,
    style_kwds=dict(color="black"),
    tooltip='NAME',
    popup=True,
    column='NAME',
    cmap='tab20',
    name='Associations',
    legend=True
)

# Adding high school zoning for Edmonds School District
hs_geo = gpd.read_file(path + "/Edmonds High Schools.json")
hs_geo.explore(
    m=assoc_map,
    style_kwds=dict(color="black"),
    tooltip='NAME',
    popup=True,
    column='NAME',
    cmap='tab10',
    name='Edmonds High Schools',
    legend=False,
    show=False
)

sysa_prop = gpd.read_file(path + "/SYSA_Proposal.json")
sysa_prop.explore(
    m=assoc_map,
    style_kwds=dict(color="black"),
    tooltip='NAME',
    popup=True,
    column='NAME',
    cmap='tab10',
    name='Proposal',
    legend=False,
    show=False
)

# Adding marker layer for all fields
field_geo = gpd.read_file(path + '/Field_Loc_Data.csv')
field_geo['geometry'] = field_geo.apply(make_point, axis=1)
field_geo.set_crs(crs='epsg:4326', inplace=True)

folium.GeoJson(
    field_geo,
    name="All Fields",
    zoom_on_click=True,
    marker=folium.Marker(icon=folium.Icon(icon='star')),
    tooltip=folium.GeoJsonTooltip(fields=["Name"]),
    popup=folium.GeoJsonPopup(fields=["Name"]),
    style_function = lambda x: {
        "markerColor": sound_FC_using(x['properties']['Name'])
    },
    show=False
).add_to(assoc_map)

# Adding marker layer for only fields in Edmonds
field_geo = gpd.read_file(path + '/Field_Loc_Data_Edmonds.csv')
field_geo['geometry'] = field_geo.apply(make_point, axis=1)
field_geo.set_crs(crs='epsg:4326', inplace=True)

folium.GeoJson(
    field_geo,
    name="Edmonds Fields",
    zoom_on_click=True,
    marker=folium.Marker(icon=folium.Icon(icon='star')),
    tooltip=folium.GeoJsonTooltip(fields=["Name"]),
    popup=folium.GeoJsonPopup(fields=["Name"]),
    style_function = lambda x: {
        "markerColor": sound_FC_using(x['properties']['Name'])
    },
    show=False
).add_to(assoc_map)

# Adds layer control to the map for toggling visibility
folium.LayerControl().add_to(assoc_map)

assoc_map

Output hidden; open in https://colab.research.google.com to view.

In [43]:
# Write to html file for download / embedding
html_rep = assoc_map._repr_html_()
mapfile = open(os.getcwd() + "/Association_Map 2.3.html", mode='w')
mapfile.write(html_rep)
mapfile.close()